In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from konlpy.tag import Okt
import re

In [2]:
mindcafe = pd.read_csv('mindcafe.csv').iloc[:, 1:]
mindcafe.head()

,title,content
0,무슨 일을 해도 금방 지쳐버려요. 무슨 문제가 있는걸까요?,안녕하세요? 올해로 29살인 여자입니다. 저는 요즘 한 고민으로 인해 생업과 현생에...
1,NaN,"안녕하세요 임신 8개월차 26살 임산부 에요 고민 끝에 여기라도 끄적여봐요 ,, 너..."
2,"지나친 책임감, 관계 단절에 대한 불안.",저는 이 두 가지가 저의 모든 문제를 만든다고 생각해요 무슨 일이든 맡기거나 믿어주...
3,지친걸까요.. 조언 부탁드립니다..,10년이면 강산도 변한다고 하죠 그 오랜 시간동안 평범했지만 뜨겁고 예쁘게 연애했고...
4,사람들과 깊게 관계를 맺지 못합니다.,안녕하세요. 저는 삼십대 중반의 여자입니다. 저는 겉으로 봤을때 큰 문제가 없습니다...


In [3]:
mindcafe.isnull().value_counts()

title  content
True   False      3757
False  False      1023
dtype: int64

In [4]:
mindcafe.loc[:, 'content'] = mindcafe.loc[:, 'content'].apply(lambda x: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', x))

In [5]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '스럽다',
            '가기', '이나고', '나다', '젠']

In [6]:
okt = Okt()
tokenized_data = []
for sentence in mindcafe['content']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

C:\Users\Yoon\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [8]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

(4530, 100)

In [9]:
print(model.wv.most_similar("너무", topn = 100))

[('무섭다', 0.9263004064559937), ('힘들다', 0.9198182821273804), ('지치다', 0.9077169895172119), ('슬프다', 0.8756189942359924), ('괴롭다', 0.8746983408927917), ('답답하다', 0.8497722744941711), ('아프다', 0.8358474969863892), ('몸', 0.8356266021728516), ('우울하다', 0.8273714184761047), ('살기', 0.8236047625541687), ('두렵다', 0.8234457969665527), ('힘드다', 0.8184133172035217), ('짜증나다', 0.8135444521903992), ('싫다', 0.7947950959205627), ('한심하다', 0.7876741886138916), ('속상하다', 0.7861335873603821), ('요즘', 0.7817736268043518), ('보기', 0.7806580662727356), ('목말', 0.7780834436416626), ('억울하다', 0.7741385698318481), ('정말', 0.7699875235557556), ('롭고', 0.7681340575218201), ('울다', 0.7609750628471375), ('미치다', 0.7608137130737305), ('식은땀', 0.7601345777511597), ('미혼', 0.7593927979469299), ('외로워', 0.7584745287895203), ('온몸', 0.7544145584106445), ('진짜', 0.7499196529388428), ('불안하다', 0.7405626773834229), ('영양실조', 0.7390739321708679), ('밉다', 0.7372246384620667), ('전류', 0.7368424534797668), ('가슴', 0.734229564666748), ('머리', 0.7322838902473

In [10]:
model1 = Word2Vec(sentences = tokenized_data, vector_size = 300, window = 3, min_count = 5, workers = 4, sg = 0)
print(model1.wv.most_similar("너무", topn = 100))

[('힘들다', 0.9349346160888672), ('무섭다', 0.9336501955986023), ('지치다', 0.9290192127227783), ('슬프다', 0.8976140022277832), ('답답하다', 0.8968304395675659), ('아프다', 0.8927211761474609), ('몸', 0.8908146619796753), ('우울하다', 0.8896341919898987), ('괴롭다', 0.8859895467758179), ('살기', 0.8671053051948547), ('힘드다', 0.8617997765541077), ('두렵다', 0.8536602258682251), ('롭고', 0.8446210026741028), ('한심하다', 0.8442767262458801), ('짜증나다', 0.8419706225395203), ('속상하다', 0.8326456546783447), ('불안하다', 0.8256296515464783), ('고배', 0.8213730454444885), ('보기', 0.8119115829467773), ('싫다', 0.8097503781318665), ('요즘', 0.8085983991622925), ('정말', 0.8012279272079468), ('머리', 0.8007693290710449), ('미치다', 0.791467547416687), ('가슴', 0.7896007895469666), ('외로워', 0.7888142466545105), ('느껴지다', 0.7866221070289612), ('울다', 0.7817863821983337), ('온몸', 0.7797648906707764), ('춥다', 0.7787177562713623), ('힘', 0.7784062623977661), ('듣기', 0.7782227993011475), ('오락가락', 0.7768094539642334), ('식은땀', 0.7760219573974609), ('억울하다', 0.774736464023

In [11]:
model2 = Word2Vec(sentences = tokenized_data, vector_size = 300, window = 2, min_count = 5, workers = 4, sg = 0)
print(model2.wv.most_similar("너무", topn = 100))

[('지치다', 0.9112237095832825), ('몸', 0.8960907459259033), ('괴롭다', 0.8956936001777649), ('무섭다', 0.8866997361183167), ('답답하다', 0.8862090706825256), ('살기', 0.8822470903396606), ('힘들다', 0.871290385723114), ('슬프다', 0.8702356219291687), ('우울하다', 0.8614525198936462), ('아프다', 0.8568299412727356), ('보기', 0.8459841012954712), ('두렵다', 0.8442648649215698), ('불안하다', 0.8375901579856873), ('짜증나다', 0.8360809683799744), ('속상하다', 0.8316767811775208), ('한심하다', 0.8312334418296814), ('힘드다', 0.83079993724823), ('고배', 0.8250585794448853), ('목말', 0.8198849558830261), ('듣기', 0.8097456693649292), ('억울하다', 0.806340754032135), ('싫다', 0.8055315613746643), ('가슴', 0.8047303557395935), ('정말', 0.8039721846580505), ('끔찍하다', 0.7987056970596313), ('너무나도', 0.7977943420410156), ('외로워', 0.7948277592658997), ('서럽다', 0.7937789559364319), ('밉다', 0.7931248545646667), ('온몸', 0.7910889387130737), ('요즘', 0.7897340655326843), ('롭고', 0.7889569401741028), ('미치다', 0.7840114831924438), ('진짜', 0.7833957076072693), ('피곤하다', 0.782410442829

In [12]:
model3 = Word2Vec(sentences = tokenized_data, vector_size = 500, window = 7, min_count = 5, workers = 6, sg = 0)
print(model3.wv.most_similar("너무", topn = 100))

[('힘들다', 0.9788833856582642), ('무섭다', 0.9646971225738525), ('지치다', 0.8898022174835205), ('싫다', 0.8892965316772461), ('아프다', 0.8812955617904663), ('슬프다', 0.8676968812942505), ('괴롭다', 0.8670147061347961), ('답답하다', 0.8635475635528564), ('힘드다', 0.8421686887741089), ('우울하다', 0.8390738368034363), ('두렵다', 0.8351356983184814), ('영양실조', 0.8234184980392456), ('살기', 0.8215354681015015), ('몸', 0.8164757490158081), ('롭고', 0.8096417188644409), ('짜증나다', 0.8093981146812439), ('식은땀', 0.7932605743408203), ('한심하다', 0.7882608771324158), ('진짜', 0.7837101221084595), ('미치다', 0.7801634669303894), ('정말', 0.772119402885437), ('온몸', 0.7703272104263306), ('울다', 0.7702916264533997), ('힘', 0.770221471786499), ('어떡하다', 0.7684500813484192), ('요즘', 0.765785276889801), ('불안하다', 0.7650703191757202), ('머리', 0.748772144317627), ('전류', 0.7481288909912109), ('죽다', 0.7424629330635071), ('도해', 0.7420429587364197), ('하아', 0.7390156984329224), ('전기', 0.7349165081977844), ('무능하다', 0.7345438599586487), ('고통스럽다', 0.733736515045166

In [13]:
model4 = Word2Vec(sentences = tokenized_data, vector_size = 500, window = 10, min_count = 5, workers = 10, sg = 0)

In [15]:
print(model4.wv.most_similar("집중", topn = 200))

[('몽롱', 0.9803446531295776), ('붙들다', 0.9767144322395325), ('당장', 0.9701233506202698), ('일상생활', 0.9682323932647705), ('머무르다', 0.9673559665679932), ('멈추다', 0.9671991467475891), ('해지', 0.9671053886413574), ('자리다', 0.9657154083251953), ('나락', 0.9655788540840149), ('무의미하다', 0.9649862051010132), ('수십', 0.9646211862564087), ('식욕', 0.964351236820221), ('흐르다', 0.9634695649147034), ('소화도', 0.9621928930282593), ('정리', 0.9616542458534241), ('동도', 0.9593068361282349), ('깜깜', 0.9590389728546143), ('움직이다', 0.9587608575820923), ('깊숙하다', 0.9580713510513306), ('매', 0.9575076103210449), ('허우적대다', 0.9570352435112), ('시도', 0.956944465637207), ('한가하다', 0.9555017948150635), ('수백', 0.9546650648117065), ('아이러니', 0.9546442031860352), ('뻗다', 0.9538180828094482), ('매다', 0.9538028240203857), ('커지다', 0.9537443518638611), ('낭비하다', 0.9536283612251282), ('흘러가다', 0.9530596137046814), ('초조하다', 0.9530518651008606), ('반복', 0.9528733491897583), ('도저히', 0.9527654647827148), ('버티고', 0.9525822997093201), ('헤치다', 0.95202690362